In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import plotly as py

In [2]:
res=requests.get('https://zh.moegirl.org/%E6%96%99%E7%90%86%E6%AC%A1%E5%85%83/%E4%B8%BB%E9%A3%9F')
soup=BeautifulSoup(res.text,'html.parser')

In [3]:
names=[]
rows=soup.tbody.find_all('tr')
for row in rows:
    all_tds=[td for td in row]
    name=all_tds[3].text
    names.append(name)
names=names[1:]

names = names[:10]
print(names)

['馒头', '白米饭', '水饺', '扬州炒饭', '热香饼', '墨西哥卷饼', '龙虾意面', '蛋包饭', '方包', '干炒牛河']


In [4]:
figure_urls=[]
for i in range(10):
    figure_url='https://zh.moegirl.org/料理次元:'+str(names[i])
    figure_urls.append(figure_url)
print(figure_urls)

['https://zh.moegirl.org/料理次元:馒头', 'https://zh.moegirl.org/料理次元:白米饭', 'https://zh.moegirl.org/料理次元:水饺', 'https://zh.moegirl.org/料理次元:扬州炒饭', 'https://zh.moegirl.org/料理次元:热香饼', 'https://zh.moegirl.org/料理次元:墨西哥卷饼', 'https://zh.moegirl.org/料理次元:龙虾意面', 'https://zh.moegirl.org/料理次元:蛋包饭', 'https://zh.moegirl.org/料理次元:方包', 'https://zh.moegirl.org/料理次元:干炒牛河']


In [5]:
# Image(url=figure_pics[i], width=100, height=100)
figure_pics=[]
session = requests.Session()
for i in range(10):
    resq=session.get(figure_urls[i], stream=True)
    soupq=BeautifulSoup(resq.text,'html.parser')
    #link=soupq.tbody.img['src']
    link=str(soupq.tbody.findAll('img')).split()
    figure_pics.append(link[-3])
    print(names[i]," : ", link[-3])

馒头  :  https://img.moegirl.org.cn/common/thumb/3/3f/%E6%96%99%E7%90%86%E6%AC%A1%E5%85%83%E9%A6%92%E5%A4%B41.png/560px-%E6%96%99%E7%90%86%E6%AC%A1%E5%85%83%E9%A6%92%E5%A4%B41.png
白米饭  :  https://img.moegirl.org.cn/common/d/d3/62390162_p0.jpg
水饺  :  https://img.moegirl.org.cn/common/thumb/6/6d/%E6%96%99%E7%90%86%E6%B0%B4%E9%A5%BA2.png/560px-%E6%96%99%E7%90%86%E6%B0%B4%E9%A5%BA2.png
扬州炒饭  :  https://img.moegirl.org.cn/common/thumb/e/e8/%E6%96%99%E7%90%86%E6%89%AC%E5%B7%9E%E7%82%92%E9%A5%AD1.png/560px-%E6%96%99%E7%90%86%E6%89%AC%E5%B7%9E%E7%82%92%E9%A5%AD1.png
热香饼  :  https://img.moegirl.org.cn/common/thumb/0/0c/%E6%96%99%E7%90%86%E7%83%AD%E9%A6%99%E9%A5%BC1a.png/560px-%E6%96%99%E7%90%86%E7%83%AD%E9%A6%99%E9%A5%BC1a.png
墨西哥卷饼  :  https://img.moegirl.org.cn/common/thumb/2/2e/%E6%96%99%E7%90%86%E5%A2%A8%E8%A5%BF%E5%93%A5%E5%8D%B7%E9%A5%BC1.png/560px-%E6%96%99%E7%90%86%E5%A2%A8%E8%A5%BF%E5%93%A5%E5%8D%B7%E9%A5%BC1.png
龙虾意面  :  https://img.moegirl.org.cn/common/thumb/2/23/%E5%BE%AE%E4%BF%A1_%E

In [7]:
def count(level):
    S=A=B=C=D=0
    for i in range(len(level)):
        if level[i]=='S':
            S+=1
        elif level[i]=='A':
            A+=1
        elif level[i]=='B':
            B+=1
        elif level[i]=='C':
            C+=1
        elif level[i]=='D':
            D+=1
    list=[S,A,B,C,D]
    return list

Level=[]
session = requests.Session()
for k in range(10):  
    level=[]
    #tds
    res=session.get(figure_urls[k], stream=True)
    soup=BeautifulSoup(res.text,'html.parser')
    rows=soup.find('table','wikitable').tbody.find_all('td')
    tds=[]
    for row in rows:
        td=[s for s in row.stripped_strings]
        tds.append(td)
    for i in range(5,26,2):
        if (i is 15)or (i is 19) or(i is 21):
            continue
        else:
            a=(str)(tds[i])
            result=''.join(re.findall(r'[A-Za-z]',a)) 
            level.append(result)
    print("Done:",k)
    
    rank=count(level)
    Level.append(rank)


Done: 0
Done: 1
Done: 2
Done: 3
Done: 4
Done: 5
Done: 6
Done: 7
Done: 8
Done: 9


In [9]:
# Handle the figure whose website is in special format
session = requests.Session()
for k in range(10):  
    if sum(Level[k]) == 0: # no elements
      res=session.get(figure_urls[k], stream=True)
      soup=BeautifulSoup(res.text,'html.parser')
      rows = soup.find_all("div", ["cyll-rightbox"])
      tmp_rank =[]
      for row in rows:
        for s in row.stripped_strings:
          result=''.join(re.findall(r'[A-Za-z]',s))
          tmp_rank.append(result)
      rank=count(tmp_rank)
      Level[k]=rank

In [10]:
Level = np.array(Level)
print(Level)

[[0 0 1 6 1]
 [0 0 2 3 1]
 [1 1 3 2 1]
 [0 2 2 3 1]
 [2 1 3 1 1]
 [1 0 4 3 0]
 [2 0 3 3 0]
 [1 0 3 3 1]
 [0 1 3 2 2]
 [0 3 1 4 0]]


In [11]:
import plotly.offline as py
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot
py.init_notebook_mode(connected=True)
pio.renderers.default = 'colab'

character=names

fig = go.Figure(go.Bar(x=character, y=Level[:,4], marker_color = "rgba(101, 101, 101, 0.6)", name='D'))
fig.add_trace(go.Bar(x=character, y=Level[:,3], marker_color = "rgba(182, 194, 154, 1)", name='C'))
fig.add_trace(go.Bar(x=character, y=Level[:,2], marker_color = "rgba(62, 111, 176, 0.6)", name='B'))
fig.add_trace(go.Bar(x=character, y=Level[:,1], marker_color = "rgba(132, 118, 202, 0.6)", name='A'))
fig.add_trace(go.Bar(x=character, y=Level[:,0], marker_color = "rgba(226, 74, 51, 0.6)", name='S'))

fig.update_layout(barmode='stack')
fig.show()